## main > utils

In [ ]:
# ['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']
# 시군구 대전데이터 한정으로, 군, 동 정보만 남긴다.
if '시군구' in X_df.columns:
  # 군, 구(동)
  X_df[['군', '구(동)']] = X_df['시군구'].str.split(' ', expand=True)[[1,2]]
  X_df.drop(columns=['시군구'], inplace=True)
# 사고일시 yyyy-mm-dd hh datetime - 시간 컬럼 생성
if '사고일시' in X_df.columns:
  # X_df['년도'] = X_df['사고일시'].str.split('-').str[0].astype(int)
  X_df['월'] = X_df['사고일시'].str.split('-').str[1].astype(int)
  X_df['시간'] = X_df['사고일시'].str.split(' ').str[-1].astype(int)
  X_df.drop(columns=['사고일시'], inplace=True)
## 버스 데이터 머지
# '군', '요일'별 버스 승하차 시간
bus = pd.read_csv("./data/origin/bus_riders_datetime.csv")
bus = bus.drop(columns=['Unnamed: 0', '동', '일자', '시'], axis=1)
bus = bus.rename(columns={'구/군':'군', '구분':'승하차'})
bus_gun = pd.DataFrame(bus.groupby(["군", "요일", "승하차"])[['05시', '06시', '07시', '08시', '09시', 
                                                      '10시', '11시', '12시', '13시', '14시', '15시', 
                                                      '16시', '17시', '18시', '19시', '20시', '21시', 
                                                      '22시', '23시','합계']].sum()).reset_index()
bus_gun_in_index = bus_gun[bus_gun['승하차'] == '승차'].index
bus_gun_out_index = bus_gun[bus_gun['승하차'] == '하차'].index
bus_gun_in = bus_gun.drop(bus_gun_out_index)
bus_gun_out = bus_gun.drop(bus_gun_in_index)
bus_gun_in.rename(columns={'05시':'승차_05시', '06시':'승차_06시', '07시':'승차_07시', '08시':'승차_08시', 
                          '09시':'승차_09시', '10시':'승차_10시', '11시':'승차_11시', '12시':'승차_12시', 
                          '13시':'승차_13시', '14시':'승차_14시', '15시':'승차_15시', '16시':'승차_16시', 
                          '17시':'승차_17시', '18시':'승차_18시', '19시':'승차_19시', '20시':'승차_20시', 
                          '21시':'승차_21시', '22시':'승차_22시', '23시':'승차_23시', '합계':'승차_합계'}, inplace=True)
bus_gun_in.drop(columns=['승하차'], inplace=True)
bus_gun_out.rename(columns={'05시':'하차_05시', '06시':'하차_06시', '07시':'하차_07시', '08시':'하차_08시', 
                            '09시':'하차_09시', '10시':'하차_10시', '11시':'하차_11시', '12시':'하차_12시', 
                            '13시':'하차_13시', '14시':'하차_14시', '15시':'하차_15시', '16시':'하차_16시', 
                            '17시':'하차_17시', '18시':'하차_18시', '19시':'하차_19시', '20시':'하차_20시', 
                            '21시':'하차_21시', '22시':'하차_22시', '23시':'하차_23시', '합계':'하차_합계'}, inplace=True)
bus_gun_out.drop(columns=['승하차'], inplace=True)
bus_gun.drop(columns=['05시', '06시', '07시', '08시', '09시', '10시', '11시',
     '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시',
     '22시', '23시', '합계','승하차'], inplace=True)
bus_gun = pd.merge(bus_gun, bus_gun_in, how='left', on=["요일", "군"])
bus_gun = pd.merge(bus_gun, bus_gun_out, how='left', on=["요일", "군"])
bus_gun = bus_gun.drop_duplicates().reset_index(drop=True)
X_df = pd.merge(X_df, bus_gun, how='left', on=['군', '요일']).set_index(X_df.index)
# '군'별 버스 노선 수
bus_noseon = pd.read_csv("./data/origin/bus_station_bus_count.csv")
bus_noseon = bus_noseon.drop(columns=["Unnamed: 0", '동'], axis=1)
bus_noseon.rename(columns={'구/군':'군'}, inplace=True)
bus_noseon = pd.DataFrame(bus_noseon.groupby("군")["경유노선수"].sum())
X_df = pd.merge(X_df, bus_noseon, how='left', on=["군"]).set_index(X_df.index)

# 도로형태 e.g. 교차로 - 교차로안 -> 도로형태와 세부도로형태로 분리
  # 전국 도로형태별 2019 ~ 2021 치사율
if '도로형태' in X_df.columns:
  X_df[['도로형태-1', '도로형태-2']] = X_df['도로형태'].str.split(' - ', expand=True)
  X_df.drop(columns=['도로형태'], inplace=True)

  print(X_df)

if '노면상태' in X_df.columns:
  # 건조 젖음/습기 기타 서리/결빙 침수 적설
  X_df['노면상태'] = X_df['노면상태'].map(lambda x: 
    0 if x == '건조' else
    1 if x == '기타' else
    2 if x == '젖음/습기' else
    3 if x == '서리/결빙' else
    4
  )

if '기상상태' in X_df.columns:
  # 맑음 비 흐림 기타 눈
  X_df['기상상태'] = X_df['기상상태'].map(lambda x: 
    0 if x == '맑음' else
    1 if x == '기타' else
    2 if x == '흐림' else
    3 if x == '비' else
    4
  )

if '요일' in X_df.columns:
  # 월 화 수 목 금 토 일
  X_df['요일'] = X_df['요일'].map(lambda x: 
    0 if x == '월요일' else
    1 if x == '화요일' else
    2 if x == '수요일' else
    3 if x == '목요일' else
    4 if x == '금요일' else
    5 if x == '토요일' else
    6
  )

return X_df

In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
train_origin = copy.deepcopy(pd.read_csv("./data/train_pp.csv"))
test_origin = copy.deepcopy(pd.read_csv("./data/test_pp.csv"))

In [3]:
train = copy.deepcopy(train_origin)
test = copy.deepcopy(test_origin)
train = train[['요일', '기상상태', '도로형태', '노면상태', '사고유형', 'month', 'year', 'time', 'gungu', 'dong','사망자수','중상자수','경상자수','부상자수','ECLO']]
test = test[['요일', '기상상태', '도로형태', '노면상태', '사고유형', 'month', 'year', 'time', 'gungu', 'dong']]
train.head()

,요일,기상상태,도로형태,노면상태,사고유형,month,year,time,gungu,dong,사망자수,중상자수,경상자수,부상자수,ECLO
0,화요일,맑음,단일로 - 기타,건조,차대사람,1,2019,0,중구,대신동,0,1,0,0,5
1,화요일,흐림,단일로 - 기타,건조,차대사람,1,2019,0,달서구,감삼동,0,0,1,0,3
2,화요일,맑음,단일로 - 기타,건조,차대사람,1,2019,1,수성구,두산동,0,0,1,0,3
3,화요일,맑음,단일로 - 기타,건조,차대차,1,2019,2,북구,복현동,0,1,0,0,5
4,화요일,맑음,단일로 - 기타,건조,차대차,1,2019,4,동구,신암동,0,0,1,0,3


In [4]:
bus = pd.read_csv("./data/bus_riders_datetime.csv")
bus = bus.drop(columns=["Unnamed: 0", "일자", "시"])
bus

,구분,05시,06시,07시,08시,09시,10시,11시,12시,13시,...,18시,19시,20시,21시,22시,23시,합계,구/군,동,요일
0,하차,0,0,1,0,0,0,0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,북구,검단동,월요일
1,하차,0,0,1,1,0,0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.0,달성군,구지면,월요일
2,승차,0,0,0,0,2,0,1,0.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,7.0,달성군,구지면,월요일
3,하차,0,0,0,0,0,0,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2.0,달서구,용산1동,월요일
4,승차,0,1,0,0,1,0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,5.0,달서구,용산1동,월요일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8363647,하차,0,1,0,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,남구,이천동,금요일
8363648,승차,0,0,0,3,0,1,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,달서구,신당동,금요일
8363649,하차,0,0,0,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,달서구,신당동,금요일
8363650,승차,0,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,달서구,신당동,금요일


In [5]:
bus_noseon = pd.read_csv("./data/bus_station_bus_count.csv")
bus_noseon = bus_noseon.drop(columns=["Unnamed: 0"])
bus_noseon.rename(columns={"구/군":"gungu", '동':'dong'}, inplace=True)
bus_noseon

,gungu,dong,경유노선수
0,남구,대명10동,61
1,남구,대명11동,40
2,남구,대명1동,18
3,남구,대명2동,33
4,남구,대명3동,69
...,...,...,...
138,중구,동인3가동,6
139,중구,삼덕동,37
140,중구,성내1동,110
141,중구,성내2동,127


In [6]:
# bus_gugun = pd.DataFrame(bus.groupby("구/군")["요일"].count())
# bus_gugun.rename(columns={"요일":"요일별통행량"}, inplace=True)
# bus_gugun

In [7]:
bus_dong = pd.DataFrame(bus.groupby(["동", "요일", "구분"])[['05시', '06시', '07시', '08시', '09시', 
                                                        '10시', '11시', '12시', '13시', '14시', '15시', 
                                                        '16시', '17시', '18시', '19시', '20시', '21시', 
                                                        '22시', '23시','합계']].sum()).reset_index()
bus_dong.rename(columns={"구분":"승하차", "동":"dong"}, inplace=True)
bus_dong

,dong,요일,승하차,05시,06시,07시,08시,09시,10시,11시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,합계
0,가창면,금요일,승차,3456,9136,31710,34154,32675,28802,24988,...,37361.0,38030.0,42961.0,45271.0,18875.0,13821.0,11694.0,3973.0,157.0,467154.0
1,가창면,금요일,하차,261,1647,3680,6694,7438,7616,8067,...,7206.0,7144.0,6186.0,5135.0,3656.0,2191.0,1867.0,1420.0,183.0,91538.0
2,가창면,목요일,승차,3400,9743,32751,34010,32106,27186,24229,...,34442.0,34644.0,34290.0,27486.0,16145.0,12135.0,8660.0,3300.0,143.0,419594.0
3,가창면,목요일,하차,254,1765,3863,6548,7066,6077,5996,...,6390.0,6697.0,5786.0,4641.0,3240.0,1969.0,1699.0,1315.0,198.0,80993.0
4,가창면,수요일,승차,3494,9830,32324,34299,31727,28758,24523,...,33902.0,33486.0,34357.0,29056.0,16144.0,12278.0,8928.0,3365.0,94.0,425537.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,효목2동,일요일,하차,1042,4158,5050,7846,10514,11895,10205,...,12770.0,12159.0,11759.0,11288.0,9527.0,7527.0,6250.0,4769.0,621.0,165404.0
2054,효목2동,토요일,승차,5567,11689,18798,31002,36827,35658,35768,...,32574.0,30759.0,31380.0,30152.0,20009.0,17207.0,16211.0,11168.0,898.0,477286.0
2055,효목2동,토요일,하차,1605,6103,8659,13839,15827,15875,15755,...,16097.0,15990.0,16143.0,15210.0,11054.0,8641.0,7761.0,6080.0,975.0,226817.0
2056,효목2동,화요일,승차,7571,20027,56132,66641,44166,36851,34605,...,41203.0,37447.0,41784.0,55091.0,25518.0,19488.0,17941.0,10905.0,717.0,626103.0


In [8]:
bus_dong

,dong,요일,승하차,05시,06시,07시,08시,09시,10시,11시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,합계
0,가창면,금요일,승차,3456,9136,31710,34154,32675,28802,24988,...,37361.0,38030.0,42961.0,45271.0,18875.0,13821.0,11694.0,3973.0,157.0,467154.0
1,가창면,금요일,하차,261,1647,3680,6694,7438,7616,8067,...,7206.0,7144.0,6186.0,5135.0,3656.0,2191.0,1867.0,1420.0,183.0,91538.0
2,가창면,목요일,승차,3400,9743,32751,34010,32106,27186,24229,...,34442.0,34644.0,34290.0,27486.0,16145.0,12135.0,8660.0,3300.0,143.0,419594.0
3,가창면,목요일,하차,254,1765,3863,6548,7066,6077,5996,...,6390.0,6697.0,5786.0,4641.0,3240.0,1969.0,1699.0,1315.0,198.0,80993.0
4,가창면,수요일,승차,3494,9830,32324,34299,31727,28758,24523,...,33902.0,33486.0,34357.0,29056.0,16144.0,12278.0,8928.0,3365.0,94.0,425537.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,효목2동,일요일,하차,1042,4158,5050,7846,10514,11895,10205,...,12770.0,12159.0,11759.0,11288.0,9527.0,7527.0,6250.0,4769.0,621.0,165404.0
2054,효목2동,토요일,승차,5567,11689,18798,31002,36827,35658,35768,...,32574.0,30759.0,31380.0,30152.0,20009.0,17207.0,16211.0,11168.0,898.0,477286.0
2055,효목2동,토요일,하차,1605,6103,8659,13839,15827,15875,15755,...,16097.0,15990.0,16143.0,15210.0,11054.0,8641.0,7761.0,6080.0,975.0,226817.0
2056,효목2동,화요일,승차,7571,20027,56132,66641,44166,36851,34605,...,41203.0,37447.0,41784.0,55091.0,25518.0,19488.0,17941.0,10905.0,717.0,626103.0


In [9]:
train_bus = pd.merge(train, bus_dong, how='inner', on=["요일", "dong"])
train_bus

,요일,기상상태,도로형태,노면상태,사고유형,month,year,time,gungu,dong,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,합계
0,화요일,맑음,단일로 - 기타,건조,차대사람,1,2019,0,중구,대신동,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
1,화요일,맑음,단일로 - 기타,건조,차대사람,1,2019,0,중구,대신동,...,4562.0,4253.0,4089.0,4985.0,2901.0,1862.0,1757.0,1424.0,179.0,59350.0
2,화요일,맑음,단일로 - 터널,건조,차대사람,1,2019,17,중구,대신동,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
3,화요일,맑음,단일로 - 터널,건조,차대사람,1,2019,17,중구,대신동,...,4562.0,4253.0,4089.0,4985.0,2901.0,1862.0,1757.0,1424.0,179.0,59350.0
4,화요일,맑음,교차로 - 교차로횡단보도내,건조,차대차,6,2019,5,중구,대신동,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18333,일요일,맑음,기타 - 기타,건조,차대차,11,2021,18,수성구,삼덕동,...,10271.0,9939.0,9691.0,9161.0,6909.0,5534.0,5220.0,4097.0,806.0,127985.0
18334,일요일,맑음,단일로 - 기타,건조,차대사람,12,2021,7,수성구,삼덕동,...,29120.0,28024.0,27362.0,31156.0,23296.0,22280.0,27201.0,25380.0,2148.0,366900.0
18335,일요일,맑음,단일로 - 기타,건조,차대사람,12,2021,7,수성구,삼덕동,...,10271.0,9939.0,9691.0,9161.0,6909.0,5534.0,5220.0,4097.0,806.0,127985.0
18336,화요일,맑음,단일로 - 기타,건조,차대차,11,2020,10,수성구,삼덕동,...,54044.0,58741.0,68382.0,69952.0,39364.0,32270.0,38896.0,35363.0,2374.0,681074.0


In [10]:
train_bus.select_dtypes(include=['number'])

,month,year,time,사망자수,중상자수,경상자수,부상자수,ECLO,05시,06시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,합계
0,1,2019,0,0,1,0,0,5,1875,2971,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
1,1,2019,0,0,1,0,0,5,378,1179,...,4562.0,4253.0,4089.0,4985.0,2901.0,1862.0,1757.0,1424.0,179.0,59350.0
2,1,2019,17,0,0,1,0,3,1875,2971,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
3,1,2019,17,0,0,1,0,3,378,1179,...,4562.0,4253.0,4089.0,4985.0,2901.0,1862.0,1757.0,1424.0,179.0,59350.0
4,6,2019,5,1,0,0,0,10,1875,2971,...,31934.0,33290.0,33513.0,32613.0,13492.0,7510.0,7652.0,5002.0,355.0,304718.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18333,11,2021,18,0,0,2,0,6,637,2848,...,10271.0,9939.0,9691.0,9161.0,6909.0,5534.0,5220.0,4097.0,806.0,127985.0
18334,12,2021,7,0,1,0,0,5,2064,6352,...,29120.0,28024.0,27362.0,31156.0,23296.0,22280.0,27201.0,25380.0,2148.0,366900.0
18335,12,2021,7,0,1,0,0,5,637,2848,...,10271.0,9939.0,9691.0,9161.0,6909.0,5534.0,5220.0,4097.0,806.0,127985.0
18336,11,2020,10,0,0,1,0,3,2277,8579,...,54044.0,58741.0,68382.0,69952.0,39364.0,32270.0,38896.0,35363.0,2374.0,681074.0


In [11]:
train_bus = pd.merge(train_bus, bus_noseon, how='inner', on=["dong", "gungu"])
pd.DataFrame(train_bus.isnull().sum())

,0
요일,0
기상상태,0
도로형태,0
노면상태,0
사고유형,0
month,0
year,0
time,0
gungu,0
dong,0


# test cumstom_X_preprocess_cat

In [12]:
# import sys
# sys.path.insert(0, '..')

# from main.datasets.utils import custom_X_preprocess_cat
# from main.datasets import HomeData
# from main.configs.preprocess_config import config

In [13]:
# home = HomeData(**config.get('input_data'), **config.get('options'))

In [14]:
import copy
import pandas as pd
import numpy as np

train_origin = copy.deepcopy(pd.read_csv("../../main/data/origin/train.csv"))
test_origin = copy.deepcopy(pd.read_csv("../../main/data/origin/test.csv"))

In [15]:
X_df = copy.deepcopy(train_origin)
X_df = X_df[['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형','ECLO']]

# ['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']
# 시군구 대전데이터 한정으로, 군, 동 정보만 남긴다.
if '시군구' in X_df.columns:
  # 군, 구(동)
  X_df[['군', '구(동)']] = X_df['시군구'].str.split(' ', expand=True)[[1,2]]
  X_df.drop(columns=['시군구'], inplace=True)

# 사고일시 yyyy-mm-dd hh datetime - 시간 컬럼 생성
if '사고일시' in X_df.columns:
  # X_df['년도'] = X_df['사고일시'].str.split('-').str[0].astype(int)
  X_df['월'] = X_df['사고일시'].str.split('-').str[1].astype(int)
  X_df['시간'] = X_df['사고일시'].str.split(' ').str[-1].astype(int)
  X_df.drop(columns=['사고일시'], inplace=True)

In [16]:
## Numeric
# df_num_cols = X_df.select_dtypes(include=['number']).columns
# X_df[df_num_cols].fillna(2, inplace=True)
# print(X_df.isna().any(axis=1).value_counts())
# df_num.fillna(3, inplace=True)
# X_df

In [17]:
# bus data merge
bus = pd.read_csv("../../main/data/origin/bus_riders_datetime.csv")
bus = bus.drop(columns=['Unnamed: 0', '동', '일자', '시'], axis=1)

In [18]:
bus = bus.rename(columns={'구/군':'군', '구분':'승하차'})
bus

,승하차,05시,06시,07시,08시,09시,10시,11시,12시,13시,...,17시,18시,19시,20시,21시,22시,23시,합계,군,요일
0,하차,0,0,1,0,0,0,0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,북구,월요일
1,하차,0,0,1,1,0,0,0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,달성군,월요일
2,승차,0,0,0,0,2,0,1,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,달성군,월요일
3,하차,0,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,달서구,월요일
4,승차,0,1,0,0,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,달서구,월요일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8363647,하차,0,1,0,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,남구,금요일
8363648,승차,0,0,0,3,0,1,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,달서구,금요일
8363649,하차,0,0,0,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,달서구,금요일
8363650,승차,0,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,달서구,금요일


In [19]:
X_city = X_df['군'].unique()
print(X_df['군'].nunique())
X_city

8


array(['중구', '달서구', '수성구', '북구', '동구', '서구', '달성군', '남구'], dtype=object)

In [20]:
bus_city = bus['군'].unique()
print(bus['군'].nunique())
bus_city

13


array(['북구', '달성군', '달서구', '중구', '수성구', '동구', '남구', '서구', '칠곡군', '경산시',
       '구미시', '고령군', '영천시'], dtype=object)

In [21]:
not_in = []
for i in range(len(X_city)):
  if X_city[i] not in bus_city:  
    not_in.append(X_city[i])
len(not_in)

0

In [22]:
bus_gun = pd.DataFrame(bus.groupby(["군", "요일", "승하차"])[['05시', '06시', '07시', '08시', '09시', 
                                                        '10시', '11시', '12시', '13시', '14시', '15시', 
                                                        '16시', '17시', '18시', '19시', '20시', '21시', 
                                                        '22시', '23시','합계']].sum()).reset_index()

In [23]:
bus_gun_in_index = bus_gun[bus_gun['승하차'] == '승차'].index
bus_gun_out_index = bus_gun[bus_gun['승하차'] == '하차'].index
bus_gun_in = bus_gun.drop(bus_gun_out_index)
bus_gun_out = bus_gun.drop(bus_gun_in_index)

bus_gun_in

,군,요일,승하차,05시,06시,07시,08시,09시,10시,11시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,합계
0,경산시,금요일,승차,26604,65993,159836,212613,160514,145113,157631,...,208453.0,231849.0,262481.0,240690.0,151424.0,125883.0,145581.0,112749.0,8317.0,3014602.0
2,경산시,목요일,승차,25545,67108,163442,219588,159400,146169,147720,...,207072.0,252074.0,275303.0,243051.0,150342.0,124777.0,154143.0,115207.0,7105.0,3014753.0
4,경산시,수요일,승차,25389,65604,162689,215687,157385,143383,147784,...,204140.0,249399.0,265917.0,245591.0,150410.0,125608.0,149728.0,113514.0,7438.0,2975226.0
6,경산시,월요일,승차,25618,66562,164966,221155,165398,145700,147139,...,205053.0,250985.0,266474.0,243917.0,147410.0,121647.0,146112.0,108622.0,7027.0,2981258.0
8,경산시,일요일,승차,16921,35649,42474,77161,99618,100910,106259,...,118798.0,126838.0,135889.0,112938.0,100655.0,93570.0,110957.0,85806.0,6749.0,1737870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,칠곡군,수요일,승차,1310,4418,10438,13663,10804,9840,9650,...,11457.0,12464.0,19000.0,20876.0,6870.0,4853.0,3732.0,1277.0,1.0,170216.0
174,칠곡군,월요일,승차,1404,4643,10878,14596,11885,10763,10297,...,11987.0,12929.0,18599.0,20565.0,6442.0,4509.0,3440.0,1142.0,24.0,175024.0
176,칠곡군,일요일,승차,1037,3756,3887,5946,7368,8804,7701,...,8820.0,9039.0,10108.0,11292.0,5499.0,4052.0,3121.0,678.0,7.0,116987.0
178,칠곡군,토요일,승차,1177,4218,6314,10354,11890,10243,11082,...,11185.0,11891.0,12731.0,12710.0,5839.0,4844.0,4009.0,1042.0,19.0,152055.0


In [24]:
bus_gun_in.rename(columns={'05시':'승차_05시', '06시':'승차_06시', '07시':'승차_07시', '08시':'승차_08시', 
                            '09시':'승차_09시', '10시':'승차_10시', '11시':'승차_11시', '12시':'승차_12시', 
                            '13시':'승차_13시', '14시':'승차_14시', '15시':'승차_15시', '16시':'승차_16시', 
                            '17시':'승차_17시', '18시':'승차_18시', '19시':'승차_19시', '20시':'승차_20시', 
                            '21시':'승차_21시', '22시':'승차_22시', '23시':'승차_23시', '합계':'승차_합계'}, inplace=True)
bus_gun_in.drop(columns=['승하차'], inplace=True)
bus_gun_in

,군,요일,승차_05시,승차_06시,승차_07시,승차_08시,승차_09시,승차_10시,승차_11시,승차_12시,...,승차_15시,승차_16시,승차_17시,승차_18시,승차_19시,승차_20시,승차_21시,승차_22시,승차_23시,승차_합계
0,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,208453.0,231849.0,262481.0,240690.0,151424.0,125883.0,145581.0,112749.0,8317.0,3014602.0
2,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,207072.0,252074.0,275303.0,243051.0,150342.0,124777.0,154143.0,115207.0,7105.0,3014753.0
4,경산시,수요일,25389,65604,162689,215687,157385,143383,147784,179323.0,...,204140.0,249399.0,265917.0,245591.0,150410.0,125608.0,149728.0,113514.0,7438.0,2975226.0
6,경산시,월요일,25618,66562,164966,221155,165398,145700,147139,177328.0,...,205053.0,250985.0,266474.0,243917.0,147410.0,121647.0,146112.0,108622.0,7027.0,2981258.0
8,경산시,일요일,16921,35649,42474,77161,99618,100910,106259,118611.0,...,118798.0,126838.0,135889.0,112938.0,100655.0,93570.0,110957.0,85806.0,6749.0,1737870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,칠곡군,수요일,1310,4418,10438,13663,10804,9840,9650,8859.0,...,11457.0,12464.0,19000.0,20876.0,6870.0,4853.0,3732.0,1277.0,1.0,170216.0
174,칠곡군,월요일,1404,4643,10878,14596,11885,10763,10297,9281.0,...,11987.0,12929.0,18599.0,20565.0,6442.0,4509.0,3440.0,1142.0,24.0,175024.0
176,칠곡군,일요일,1037,3756,3887,5946,7368,8804,7701,8162.0,...,8820.0,9039.0,10108.0,11292.0,5499.0,4052.0,3121.0,678.0,7.0,116987.0
178,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,11185.0,11891.0,12731.0,12710.0,5839.0,4844.0,4009.0,1042.0,19.0,152055.0


In [25]:
bus_gun_out.rename(columns={'05시':'하차_05시', '06시':'하차_06시', '07시':'하차_07시', '08시':'하차_08시', 
                            '09시':'하차_09시', '10시':'하차_10시', '11시':'하차_11시', '12시':'하차_12시', 
                            '13시':'하차_13시', '14시':'하차_14시', '15시':'하차_15시', '16시':'하차_16시', 
                            '17시':'하차_17시', '18시':'하차_18시', '19시':'하차_19시', '20시':'하차_20시', 
                            '21시':'하차_21시', '22시':'하차_22시', '23시':'하차_23시', '합계':'하차_합계'}, inplace=True)
# bus_gun_out.drop(columns=['군','요일','승하차'], inplace=True)
bus_gun_out.drop(columns=['승하차'], inplace=True)
bus_gun_out

,군,요일,하차_05시,하차_06시,하차_07시,하차_08시,하차_09시,하차_10시,하차_11시,하차_12시,...,하차_15시,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계
1,경산시,금요일,6101,28546,46858,62236,50357,48479,52131,58854.0,...,67423.0,73711.0,79403.0,77831.0,53496.0,38451.0,39570.0,32630.0,6291.0,945801.0
3,경산시,목요일,6020,29455,47954,64368,51711,48515,48832,56289.0,...,65222.0,71426.0,78756.0,76201.0,50150.0,35458.0,38230.0,29921.0,5029.0,919884.0
5,경산시,수요일,6213,28984,48235,63703,50350,48133,49833,56122.0,...,66480.0,71172.0,77304.0,76376.0,50050.0,35792.0,37431.0,29682.0,4952.0,916957.0
7,경산시,월요일,6402,28662,48608,64291,53927,51465,54170,59127.0,...,67642.0,72552.0,79831.0,77612.0,50251.0,34794.0,36194.0,27971.0,4686.0,939622.0
9,경산시,일요일,3601,18783,15540,23747,31214,33264,35710,38585.0,...,44398.0,47676.0,50143.0,44697.0,37195.0,32113.0,32516.0,25274.0,4640.0,605775.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,칠곡군,수요일,96,2014,1739,4277,3117,2883,2432,2194.0,...,3056.0,3105.0,2121.0,2042.0,1607.0,1401.0,839.0,382.0,37.0,38101.0
175,칠곡군,월요일,99,2046,1820,4631,3302,3137,2758,2432.0,...,3114.0,3173.0,2139.0,2113.0,1596.0,1401.0,850.0,319.0,51.0,40112.0
177,칠곡군,일요일,72,2015,894,2706,3144,2556,2076,1844.0,...,2480.0,2711.0,2221.0,2042.0,1589.0,1778.0,960.0,291.0,48.0,33799.0
179,칠곡군,토요일,94,2277,1482,3916,3909,3184,2878,2453.0,...,3194.0,3203.0,2323.0,2280.0,1627.0,1868.0,1100.0,426.0,70.0,41518.0


In [26]:
bus_gun.drop(columns=['05시', '06시', '07시', '08시', '09시', '10시', '11시',
       '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시',
       '22시', '23시', '합계','승하차'], inplace=True)

In [27]:
bus_gun = pd.merge(bus_gun, bus_gun_in, how='left', on=["요일", "군"])
bus_gun

,군,요일,승차_05시,승차_06시,승차_07시,승차_08시,승차_09시,승차_10시,승차_11시,승차_12시,...,승차_15시,승차_16시,승차_17시,승차_18시,승차_19시,승차_20시,승차_21시,승차_22시,승차_23시,승차_합계
0,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,208453.0,231849.0,262481.0,240690.0,151424.0,125883.0,145581.0,112749.0,8317.0,3014602.0
1,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,208453.0,231849.0,262481.0,240690.0,151424.0,125883.0,145581.0,112749.0,8317.0,3014602.0
2,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,207072.0,252074.0,275303.0,243051.0,150342.0,124777.0,154143.0,115207.0,7105.0,3014753.0
3,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,207072.0,252074.0,275303.0,243051.0,150342.0,124777.0,154143.0,115207.0,7105.0,3014753.0
4,경산시,수요일,25389,65604,162689,215687,157385,143383,147784,179323.0,...,204140.0,249399.0,265917.0,245591.0,150410.0,125608.0,149728.0,113514.0,7438.0,2975226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,칠곡군,일요일,1037,3756,3887,5946,7368,8804,7701,8162.0,...,8820.0,9039.0,10108.0,11292.0,5499.0,4052.0,3121.0,678.0,7.0,116987.0
178,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,11185.0,11891.0,12731.0,12710.0,5839.0,4844.0,4009.0,1042.0,19.0,152055.0
179,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,11185.0,11891.0,12731.0,12710.0,5839.0,4844.0,4009.0,1042.0,19.0,152055.0
180,칠곡군,화요일,1335,4653,10695,14192,10718,9807,9827,9322.0,...,11784.0,13618.0,19558.0,21249.0,6515.0,4613.0,3744.0,1172.0,9.0,173919.0


In [28]:
bus_gun = pd.merge(bus_gun, bus_gun_out, how='left', on=["요일", "군"])
bus_gun

,군,요일,승차_05시,승차_06시,승차_07시,승차_08시,승차_09시,승차_10시,승차_11시,승차_12시,...,하차_15시,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계
0,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,67423.0,73711.0,79403.0,77831.0,53496.0,38451.0,39570.0,32630.0,6291.0,945801.0
1,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,67423.0,73711.0,79403.0,77831.0,53496.0,38451.0,39570.0,32630.0,6291.0,945801.0
2,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,65222.0,71426.0,78756.0,76201.0,50150.0,35458.0,38230.0,29921.0,5029.0,919884.0
3,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,65222.0,71426.0,78756.0,76201.0,50150.0,35458.0,38230.0,29921.0,5029.0,919884.0
4,경산시,수요일,25389,65604,162689,215687,157385,143383,147784,179323.0,...,66480.0,71172.0,77304.0,76376.0,50050.0,35792.0,37431.0,29682.0,4952.0,916957.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,칠곡군,일요일,1037,3756,3887,5946,7368,8804,7701,8162.0,...,2480.0,2711.0,2221.0,2042.0,1589.0,1778.0,960.0,291.0,48.0,33799.0
178,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,3194.0,3203.0,2323.0,2280.0,1627.0,1868.0,1100.0,426.0,70.0,41518.0
179,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,3194.0,3203.0,2323.0,2280.0,1627.0,1868.0,1100.0,426.0,70.0,41518.0
180,칠곡군,화요일,1335,4653,10695,14192,10718,9807,9827,9322.0,...,2961.0,3200.0,2304.0,2072.0,1604.0,1519.0,936.0,373.0,37.0,39395.0


In [29]:
bus_gun = bus_gun.drop_duplicates().reset_index(drop=True)
bus_gun

,군,요일,승차_05시,승차_06시,승차_07시,승차_08시,승차_09시,승차_10시,승차_11시,승차_12시,...,하차_15시,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계
0,경산시,금요일,26604,65993,159836,212613,160514,145113,157631,199091.0,...,67423.0,73711.0,79403.0,77831.0,53496.0,38451.0,39570.0,32630.0,6291.0,945801.0
1,경산시,목요일,25545,67108,163442,219588,159400,146169,147720,182522.0,...,65222.0,71426.0,78756.0,76201.0,50150.0,35458.0,38230.0,29921.0,5029.0,919884.0
2,경산시,수요일,25389,65604,162689,215687,157385,143383,147784,179323.0,...,66480.0,71172.0,77304.0,76376.0,50050.0,35792.0,37431.0,29682.0,4952.0,916957.0
3,경산시,월요일,25618,66562,164966,221155,165398,145700,147139,177328.0,...,67642.0,72552.0,79831.0,77612.0,50251.0,34794.0,36194.0,27971.0,4686.0,939622.0
4,경산시,일요일,16921,35649,42474,77161,99618,100910,106259,118611.0,...,44398.0,47676.0,50143.0,44697.0,37195.0,32113.0,32516.0,25274.0,4640.0,605775.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,칠곡군,수요일,1310,4418,10438,13663,10804,9840,9650,8859.0,...,3056.0,3105.0,2121.0,2042.0,1607.0,1401.0,839.0,382.0,37.0,38101.0
87,칠곡군,월요일,1404,4643,10878,14596,11885,10763,10297,9281.0,...,3114.0,3173.0,2139.0,2113.0,1596.0,1401.0,850.0,319.0,51.0,40112.0
88,칠곡군,일요일,1037,3756,3887,5946,7368,8804,7701,8162.0,...,2480.0,2711.0,2221.0,2042.0,1589.0,1778.0,960.0,291.0,48.0,33799.0
89,칠곡군,토요일,1177,4218,6314,10354,11890,10243,11082,10615.0,...,3194.0,3203.0,2323.0,2280.0,1627.0,1868.0,1100.0,426.0,70.0,41518.0


In [30]:
X_df = pd.merge(X_df, bus_gun, how='left', on=['군', '요일'])
X_df

,요일,기상상태,도로형태,노면상태,사고유형,ECLO,군,구(동),월,시간,...,하차_15시,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계
0,화요일,맑음,단일로 - 기타,건조,차대사람,5,중구,대신동,1,0,...,317578.0,328170.0,337161.0,390955.0,213874.0,138111.0,128337.0,101351.0,13820.0,4312953.0
1,화요일,흐림,단일로 - 기타,건조,차대사람,3,달서구,감삼동,1,0,...,519076.0,568933.0,621974.0,706516.0,451755.0,303784.0,274212.0,205230.0,24472.0,7981991.0
2,화요일,맑음,단일로 - 기타,건조,차대사람,3,수성구,두산동,1,1,...,379703.0,434731.0,431230.0,485386.0,314585.0,222942.0,229413.0,196649.0,26566.0,5553222.0
3,화요일,맑음,단일로 - 기타,건조,차대차,5,북구,복현동,1,2,...,402412.0,444476.0,487943.0,559689.0,335661.0,224123.0,218339.0,163491.0,20101.0,5995335.0
4,화요일,맑음,단일로 - 기타,건조,차대차,3,동구,신암동,1,4,...,450347.0,491182.0,517377.0,559731.0,340969.0,229322.0,231292.0,181646.0,20960.0,6512848.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,교차로 - 교차로안,건조,차대차,3,수성구,수성동3가,12,19,...,401414.0,444284.0,450704.0,509845.0,353096.0,242058.0,248049.0,224726.0,32512.0,5722718.0
39605,금요일,맑음,단일로 - 기타,건조,차대차,3,달서구,상인동,12,19,...,550524.0,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0
39606,금요일,맑음,교차로 - 교차로안,건조,차대차,10,달서구,월성동,12,21,...,550524.0,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0
39607,금요일,맑음,기타 - 기타,건조,차대차,3,달서구,장동,12,22,...,550524.0,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0


In [31]:
bus_noseon = pd.read_csv("../../main/data/origin/bus_station_bus_count.csv")
bus_noseon = bus_noseon.drop(columns=["Unnamed: 0", '동'], axis=1)
bus_noseon.rename(columns={'구/군':'군'}, inplace=True)
bus_noseon

,군,경유노선수
0,남구,61
1,남구,40
2,남구,18
3,남구,33
4,남구,69
...,...,...
138,중구,6
139,중구,37
140,중구,110
141,중구,127


In [33]:
# X_city = X_df['구(동)'].unique()
# print(X_df['구(동)'].nunique())
# X_city
# noseon_city = bus_noseon['동'].unique()
# print(bus_noseon['동'].nunique())
# noseon_city
# not_in = []
# for i in range(len(X_city)):
#   if X_city[i] not in noseon_city:  
#     not_in.append(X_city[i])
# len(not_in)

In [34]:
bus_noseon = pd.DataFrame(bus_noseon.groupby("군")["경유노선수"].sum())
bus_noseon

,경유노선수
군,
남구,592
달서구,1930
달성군,1538
동구,2006
북구,2193
서구,743
수성구,1918
중구,607


In [35]:
X_df = pd.merge(X_df, bus_noseon, how='left', on=["군"])
X_df

,요일,기상상태,도로형태,노면상태,사고유형,ECLO,군,구(동),월,시간,...,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계,경유노선수
0,화요일,맑음,단일로 - 기타,건조,차대사람,5,중구,대신동,1,0,...,328170.0,337161.0,390955.0,213874.0,138111.0,128337.0,101351.0,13820.0,4312953.0,607
1,화요일,흐림,단일로 - 기타,건조,차대사람,3,달서구,감삼동,1,0,...,568933.0,621974.0,706516.0,451755.0,303784.0,274212.0,205230.0,24472.0,7981991.0,1930
2,화요일,맑음,단일로 - 기타,건조,차대사람,3,수성구,두산동,1,1,...,434731.0,431230.0,485386.0,314585.0,222942.0,229413.0,196649.0,26566.0,5553222.0,1918
3,화요일,맑음,단일로 - 기타,건조,차대차,5,북구,복현동,1,2,...,444476.0,487943.0,559689.0,335661.0,224123.0,218339.0,163491.0,20101.0,5995335.0,2193
4,화요일,맑음,단일로 - 기타,건조,차대차,3,동구,신암동,1,4,...,491182.0,517377.0,559731.0,340969.0,229322.0,231292.0,181646.0,20960.0,6512848.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,교차로 - 교차로안,건조,차대차,3,수성구,수성동3가,12,19,...,444284.0,450704.0,509845.0,353096.0,242058.0,248049.0,224726.0,32512.0,5722718.0,1918
39605,금요일,맑음,단일로 - 기타,건조,차대차,3,달서구,상인동,12,19,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39606,금요일,맑음,교차로 - 교차로안,건조,차대차,10,달서구,월성동,12,21,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39607,금요일,맑음,기타 - 기타,건조,차대차,3,달서구,장동,12,22,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930


In [36]:
X_df

,요일,기상상태,도로형태,노면상태,사고유형,ECLO,군,구(동),월,시간,...,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계,경유노선수
0,화요일,맑음,단일로 - 기타,건조,차대사람,5,중구,대신동,1,0,...,328170.0,337161.0,390955.0,213874.0,138111.0,128337.0,101351.0,13820.0,4312953.0,607
1,화요일,흐림,단일로 - 기타,건조,차대사람,3,달서구,감삼동,1,0,...,568933.0,621974.0,706516.0,451755.0,303784.0,274212.0,205230.0,24472.0,7981991.0,1930
2,화요일,맑음,단일로 - 기타,건조,차대사람,3,수성구,두산동,1,1,...,434731.0,431230.0,485386.0,314585.0,222942.0,229413.0,196649.0,26566.0,5553222.0,1918
3,화요일,맑음,단일로 - 기타,건조,차대차,5,북구,복현동,1,2,...,444476.0,487943.0,559689.0,335661.0,224123.0,218339.0,163491.0,20101.0,5995335.0,2193
4,화요일,맑음,단일로 - 기타,건조,차대차,3,동구,신암동,1,4,...,491182.0,517377.0,559731.0,340969.0,229322.0,231292.0,181646.0,20960.0,6512848.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,교차로 - 교차로안,건조,차대차,3,수성구,수성동3가,12,19,...,444284.0,450704.0,509845.0,353096.0,242058.0,248049.0,224726.0,32512.0,5722718.0,1918
39605,금요일,맑음,단일로 - 기타,건조,차대차,3,달서구,상인동,12,19,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39606,금요일,맑음,교차로 - 교차로안,건조,차대차,10,달서구,월성동,12,21,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39607,금요일,맑음,기타 - 기타,건조,차대차,3,달서구,장동,12,22,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930


In [37]:
# 도로형태 e.g. 교차로 - 교차로안 -> 도로형태와 세부도로형태로 분리
if '도로형태' in X_df.columns:
  X_df[['도로형태-1', '도로형태-2']] = X_df['도로형태'].str.split(' - ', expand=True)
  X_df.drop(columns=['도로형태'], inplace=True)

  print(X_df)
  # df_test 의 unique values 와 df_train 의 unique values 가 다르므로 우선 제외시킴
  # X_df.drop(columns=['구(동)'], inplace=True)

if '노면상태' in X_df.columns:
  # 건조 젖음/습기 기타 서리/결빙 침수 적설
  X_df['노면상태'] = X_df['노면상태'].map(lambda x: 
    0 if x == '건조' else
    1 if x == '기타' else
    2 if x == '젖음/습기' else
    3 if x == '서리/결빙' else
    4
  )

if '기상상태' in X_df.columns:
  # 맑음 비 흐림 기타 눈
  X_df['기상상태'] = X_df['기상상태'].map(lambda x: 
    0 if x == '맑음' else
    1 if x == '기타' else
    2 if x == '흐림' else
    3 if x == '비' else
    4
  )

if '요일' in X_df.columns:
  # 월 화 수 목 금 토 일
  X_df['요일'] = X_df['요일'].map(lambda x: 
    0 if x == '월' else
    1 if x == '화' else
    2 if x == '수' else
    3 if x == '목' else
    4 if x == '금' else
    5 if x == '토' else
    6
  )

        요일 기상상태 노면상태  사고유형  ECLO    군   구(동)   월  시간  승차_05시  ...    하차_18시  \
0      화요일   맑음   건조  차대사람     5   중구    대신동   1   0   50819  ...  390955.0   
1      화요일   흐림   건조  차대사람     3  달서구    감삼동   1   0  215876  ...  706516.0   
2      화요일   맑음   건조  차대사람     3  수성구    두산동   1   1  123802  ...  485386.0   
3      화요일   맑음   건조   차대차     5   북구    복현동   1   2  188549  ...  559689.0   
4      화요일   맑음   건조   차대차     3   동구    신암동   1   4  175256  ...  559731.0   
...    ...  ...  ...   ...   ...  ...    ...  ..  ..     ...  ...       ...   
39604  금요일   맑음   건조   차대차     3  수성구  수성동3가  12  19  121455  ...  509845.0   
39605  금요일   맑음   건조   차대차     3  달서구    상인동  12  19  215978  ...  728675.0   
39606  금요일   맑음   건조   차대차    10  달서구    월성동  12  21  215978  ...  728675.0   
39607  금요일   맑음   건조   차대차     3  달서구     장동  12  22  215978  ...  728675.0   
39608  금요일   맑음   건조   차대차     6   서구    비산동  12  23  149204  ...  340667.0   

         하차_19시    하차_20시    하차_21시    하차_22시   하차_

In [38]:
df_cat_num = X_df.select_dtypes(include=['number'])
df_cat_num

,요일,기상상태,노면상태,ECLO,월,시간,승차_05시,승차_06시,승차_07시,승차_08시,...,하차_16시,하차_17시,하차_18시,하차_19시,하차_20시,하차_21시,하차_22시,하차_23시,하차_합계,경유노선수
0,6,0,0,5,1,0,50819,128897,342472,597334,...,328170.0,337161.0,390955.0,213874.0,138111.0,128337.0,101351.0,13820.0,4312953.0,607
1,6,2,0,3,1,0,215876,707089,1973725,2141187,...,568933.0,621974.0,706516.0,451755.0,303784.0,274212.0,205230.0,24472.0,7981991.0,1930
2,6,0,0,3,1,1,123802,372314,1285778,1535249,...,434731.0,431230.0,485386.0,314585.0,222942.0,229413.0,196649.0,26566.0,5553222.0,1918
3,6,0,0,5,1,2,188549,580610,1819331,1903416,...,444476.0,487943.0,559689.0,335661.0,224123.0,218339.0,163491.0,20101.0,5995335.0,2193
4,6,0,0,3,1,4,175256,453795,1353050,1620767,...,491182.0,517377.0,559731.0,340969.0,229322.0,231292.0,181646.0,20960.0,6512848.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,6,0,0,3,12,19,121455,361477,1216213,1480993,...,444284.0,450704.0,509845.0,353096.0,242058.0,248049.0,224726.0,32512.0,5722718.0,1918
39605,6,0,0,3,12,19,215978,683685,1878657,2059657,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39606,6,0,0,10,12,21,215978,683685,1878657,2059657,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930
39607,6,0,0,3,12,22,215978,683685,1878657,2059657,...,586809.0,648456.0,728675.0,497434.0,322715.0,285679.0,225625.0,29265.0,8155751.0,1930


In [39]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일      39609 non-null  int64  
 1   기상상태    39609 non-null  int64  
 2   노면상태    39609 non-null  int64  
 3   사고유형    39609 non-null  object 
 4   ECLO    39609 non-null  int64  
 5   군       39609 non-null  object 
 6   구(동)    39609 non-null  object 
 7   월       39609 non-null  int64  
 8   시간      39609 non-null  int64  
 9   승차_05시  39609 non-null  int64  
 10  승차_06시  39609 non-null  int64  
 11  승차_07시  39609 non-null  int64  
 12  승차_08시  39609 non-null  int64  
 13  승차_09시  39609 non-null  int64  
 14  승차_10시  39609 non-null  int64  
 15  승차_11시  39609 non-null  int64  
 16  승차_12시  39609 non-null  float64
 17  승차_13시  39609 non-null  float64
 18  승차_14시  39609 non-null  float64
 19  승차_15시  39609 non-null  float64
 20  승차_16시  39609 non-null  float64
 21  승차_17시  39609 non-null  float64
 22

In [40]:
X_df.isnull().sum().value_counts()

0    52
Name: count, dtype: int64

In [41]:
df_cat = X_df.select_dtypes(include=['object'])
df_cat

,사고유형,군,구(동),도로형태-1,도로형태-2
0,차대사람,중구,대신동,단일로,기타
1,차대사람,달서구,감삼동,단일로,기타
2,차대사람,수성구,두산동,단일로,기타
3,차대차,북구,복현동,단일로,기타
4,차대차,동구,신암동,단일로,기타
...,...,...,...,...,...
39604,차대차,수성구,수성동3가,교차로,교차로안
39605,차대차,달서구,상인동,단일로,기타
39606,차대차,달서구,월성동,교차로,교차로안
39607,차대차,달서구,장동,기타,기타


In [42]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(dtype=np.float32, sparse_output=False, drop='if_binary', handle_unknown='ignore')
df_cat_onehot = pd.DataFrame(enc.fit_transform(df_cat))

In [43]:
df_cat_onehot

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,221
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39605,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
39606,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39607,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [44]:
df_cat_num.shape, df_cat_onehot.shape

((39609, 47), (39609, 222))

In [45]:
df_arr = pd.concat([df_cat_num.reset_index(drop=True), df_cat_onehot.reset_index(drop=True)], axis=1).set_index(X_df.index)
df_arr

,요일,기상상태,노면상태,ECLO,월,시간,승차_05시,승차_06시,승차_07시,승차_08시,...,212,213,214,215,216,217,218,219,220,221
0,6,0,0,5,1,0,50819,128897,342472,597334,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,6,2,0,3,1,0,215876,707089,1973725,2141187,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,0,0,3,1,1,123802,372314,1285778,1535249,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,6,0,0,5,1,2,188549,580610,1819331,1903416,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,6,0,0,3,1,4,175256,453795,1353050,1620767,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,6,0,0,3,12,19,121455,361477,1216213,1480993,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39605,6,0,0,3,12,19,215978,683685,1878657,2059657,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
39606,6,0,0,10,12,21,215978,683685,1878657,2059657,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39607,6,0,0,3,12,22,215978,683685,1878657,2059657,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [46]:
df_arr.isnull().sum().value_counts()

0    269
Name: count, dtype: int64